<h2 align="center">AtliQ Mart's Retail Events Analysis<h2>

### Project Name : Analyse Promotions and Provide Tangible Insights to Sales Director

### Project Domain: FMCG Function: Sales / Promotions

### Problem Statement :
#### AtliQ Mart is a retail giant with over 50 supermarkets in the southern region of India. All their 50 stores ran a massive promotion during the Diwali 2023 and Sankranti 2024 (festive time in India) on their AtliQ branded products. Now the sales director wants to understand which promotions did well and which did not so that they can make informed decisions for their next promotional period

In [1]:
## importing pandas library for analysis
import pandas as pd

In [2]:
## storing the data from csv files to dataframes 
df_campaigns=pd.read_csv("dataset/dim_campaigns.csv")
df_products=pd.read_csv("dataset/dim_products.csv")
df_stores=pd.read_csv("dataset/dim_stores.csv")
df_events=pd.read_csv("dataset/fact_events.csv")

## Store Performance Analysis

#### Which are the top 10 stores in terms of Incremental Revenue (IR) generated from the promotions?


In [3]:
## merging df_stores and df_events as required columns for the given question are in that dataframes
df=pd.merge(df_stores,df_events,on="store_id")
df.head(4)

,store_id,city,event_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo)
0,STTRV-0,Trivandrum,c4db5b,CAMP_DIW_01,P01,172,33% OFF,164,236
1,STTRV-0,Trivandrum,7fc923,CAMP_SAN_01,P08,1190,BOGOF,19,74
2,STTRV-0,Trivandrum,72f6b4,CAMP_SAN_01,P02,860,33% OFF,241,344
3,STTRV-0,Trivandrum,ccbc3a,CAMP_DIW_01,P06,415,25% OFF,52,49


In [4]:
## creating calculated column total_revenue_before_promo
df["total_revenue_before_promo"]=df["base_price"]*df["quantity_sold(before_promo)"]
df.head(2)

,store_id,city,event_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),total_revenue_before_promo
0,STTRV-0,Trivandrum,c4db5b,CAMP_DIW_01,P01,172,33% OFF,164,236,28208
1,STTRV-0,Trivandrum,7fc923,CAMP_SAN_01,P08,1190,BOGOF,19,74,22610


In [5]:
## finding unique promo types
df["promo_type"].unique()

array(['33% OFF', 'BOGOF', '25% OFF', '50% OFF', '500 Cashback'],
      dtype=object)

In [6]:
## this function will find revenue after promotion is applied based on promo type
def converter(promotype,base_price,quantity_after_promo):
    if promotype=="50% OFF":
        revenue=base_price*0.5*quantity_after_promo
    elif promotype=="25% OFF":
        revenue=base_price*0.75*quantity_after_promo
    elif promotype=="33% OFF":
        revenue=base_price*0.67*quantity_after_promo
    elif promotype=="500 Cashback":
        revenue=(base_price*quantity_after_promo)-500
    elif promotype=="BOGOF":
        revenue=base_price*0.5*quantity_after_promo*2
    return revenue
         

In [7]:
## finding total_revenue_before_promo
df["total_revenue_after_promo"]=df.apply(lambda x:converter(x["promo_type"],x["base_price"],x["quantity_sold(after_promo)"]),axis=1)

In [8]:
df.head(3)

,store_id,city,event_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),total_revenue_before_promo,total_revenue_after_promo
0,STTRV-0,Trivandrum,c4db5b,CAMP_DIW_01,P01,172,33% OFF,164,236,28208,27196.64
1,STTRV-0,Trivandrum,7fc923,CAMP_SAN_01,P08,1190,BOGOF,19,74,22610,88060.00
2,STTRV-0,Trivandrum,72f6b4,CAMP_SAN_01,P02,860,33% OFF,241,344,207260,198212.80


In [9]:
## incremental revenue is difference between total revenue after promo to that of total revenue before promo
df["IR"]=df["total_revenue_after_promo"]-df["total_revenue_before_promo"]

In [10]:
df.head(4)

,store_id,city,event_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),total_revenue_before_promo,total_revenue_after_promo,IR
0,STTRV-0,Trivandrum,c4db5b,CAMP_DIW_01,P01,172,33% OFF,164,236,28208,27196.64,-1011.36
1,STTRV-0,Trivandrum,7fc923,CAMP_SAN_01,P08,1190,BOGOF,19,74,22610,88060.00,65450.00
2,STTRV-0,Trivandrum,72f6b4,CAMP_SAN_01,P02,860,33% OFF,241,344,207260,198212.80,-9047.20
3,STTRV-0,Trivandrum,ccbc3a,CAMP_DIW_01,P06,415,25% OFF,52,49,21580,15251.25,-6328.75


In [11]:
## finding top 10 stores by grouping store id with Incremental revenue
top_10_stores=df.groupby("store_id")["IR"].sum().round(2).sort_values(ascending=False)

In [12]:
top_10_stores.head(10)

store_id
STMYS-1    5859313.54
STCHE-4    5760030.03
STBLR-0    5702623.65
STBLR-7    5583587.33
STCHE-7    5500225.72
STBLR-6    5460720.29
STMYS-3    5358791.98
STCHE-3    5242650.33
STCHE-6    4840345.71
STBLR-3    4748443.40
Name: IR, dtype: float64

#### Which are the bottom 10 stores when it comes to Incremental Sold Units (ISU) during the promotional period?


In [14]:
## finding incremental sold units
df["ISU"]=df["quantity_sold(after_promo)"]-df["quantity_sold(before_promo)"]

In [18]:
## finding bottom 10 stores by incremental sold units
bottom_10_stores=df.groupby("store_id")["ISU"].sum().sort_values()

In [19]:
bottom_10_stores.head(10)

store_id
STMLR-0    1952
STVSK-3    2209
STVSK-4    2469
STTRV-1    2604
STMLR-2    2664
STTRV-0    2733
STVJD-1    2763
STMLR-1    2784
STCBE-4    2927
STVJD-0    3046
Name: ISU, dtype: int64

#### What are the top 2 promotion types that resulted in the highest Incremental Revenue?


In [20]:
## as we already have Incremental revenue in our df reusing it to get top 2 promo types by Incremental revenue
top_2_promo_types=df.groupby("promo_type")["IR"].sum().sort_values(ascending=False)

In [21]:
top_2_promo_types.head(2)

promo_type
500 Cashback    122593000.0
BOGOF            69316990.0
Name: IR, dtype: float64

#### What are the bottom 2 promotion types in terms of their impact on Incremental Sold Units?

In [22]:
## finding bottom 2 promo types by Incremental revenue
bottom_2_promo_types=df.groupby("promo_type")["ISU"].sum().sort_values()

In [23]:
bottom_2_promo_types.head(2)

promo_type
25% OFF   -5717
50% OFF    6931
Name: ISU, dtype: int64

## Product and Category Analysis

#### Which product categories saw the most significant lift in sales from the promotions?

In [24]:
## merging our dataframe which already have IR and ISU with df_products to get product category wise sales
df=pd.merge(df,df_products,on="product_code")
df.head(2)

,store_id,city,event_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),total_revenue_before_promo,total_revenue_after_promo,IR,ISU,product_name,category
0,STTRV-0,Trivandrum,c4db5b,CAMP_DIW_01,P01,172,33% OFF,164,236,28208,27196.64,-1011.36,72,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,STTRV-0,Trivandrum,30c488,CAMP_SAN_01,P01,172,33% OFF,132,187,22704,21549.88,-1154.12,55,Atliq_Masoor_Dal (1KG),Grocery & Staples


In [25]:
## groupping product category by IR
result=df.groupby("category")["IR"].sum()
result

category
Combo1               1.225930e+08
Grocery & Staples    2.399289e+07
Home Appliances      2.515039e+07
Home Care            1.555468e+07
Personal Care       -8.454985e+05
Name: IR, dtype: float64

In [26]:
## as the values seem to be big , I have formatted those values in millions using below function
def format_value(value):
    if value >= 1e6:
        return '{:.2f} mln'.format(value / 1e6)
    elif value >= 1e3:
        return '{:.2f} k'.format(value / 1e3)
    else:
        return '{:.2f}'.format(value)


In [27]:
## applying the above function to the result dataframe
result=result.apply(format_value)

In [28]:
result

category
Combo1               122.59 mln
Grocery & Staples     23.99 mln
Home Appliances       25.15 mln
Home Care             15.55 mln
Personal Care        -845498.50
Name: IR, dtype: object

#### Are there specific products that respond exceptionally well or poorly to promotions?

In [29]:
## groupping the products by IR
result=df.groupby("product_name")["IR"].sum().sort_values(ascending=False)
result

product_name
Atliq_Home_Essential_8_Product_Combo         1.225930e+08
Atliq_waterproof_Immersion_Rod               1.756134e+07
Atliq_Farm_Chakki_Atta (1KG)                 1.736348e+07
Atliq_Double_Bedsheet_set                    1.291745e+07
Atliq_Suflower_Oil (1L)                      8.192769e+06
Atliq_High_Glo_15W_LED_Bulb                  7.589050e+06
Atliq_Curtains                               3.517500e+06
Atliq_Scrub_Sponge_For_Dishwash             -1.112788e+05
Atliq_Cream_Beauty_Bathing_Soap (125GM)     -1.377525e+05
Atliq_Lime_Cool_Bathing_Bar (125GM)         -1.598360e+05
Atliq_Masoor_Dal (1KG)                      -1.757032e+05
Atliq_Body_Milk_Nourishing_Lotion (120ML)   -2.161700e+05
Atliq_Doodh_Kesar_Body_Lotion (200ML)       -3.317400e+05
Atliq_Fusion_Container_Set_of_3             -7.689950e+05
Atliq_Sonamasuri_Rice (10KG)                -1.387653e+06
Name: IR, dtype: float64

In [30]:
## from above result we got there are total 15 products out of which top 7 are perfoming well
high_IR_products=result.head(7)
high_IR_products=high_IR_products.apply(format_value)
high_IR_products

product_name
Atliq_Home_Essential_8_Product_Combo    122.59 mln
Atliq_waterproof_Immersion_Rod           17.56 mln
Atliq_Farm_Chakki_Atta (1KG)             17.36 mln
Atliq_Double_Bedsheet_set                12.92 mln
Atliq_Suflower_Oil (1L)                   8.19 mln
Atliq_High_Glo_15W_LED_Bulb               7.59 mln
Atliq_Curtains                            3.52 mln
Name: IR, dtype: object

In [31]:
## there are total 15 products out of which bottom 8 are perfoming poorly
result.tail(8)

product_name
Atliq_Scrub_Sponge_For_Dishwash              -111278.75
Atliq_Cream_Beauty_Bathing_Soap (125GM)      -137752.50
Atliq_Lime_Cool_Bathing_Bar (125GM)          -159836.00
Atliq_Masoor_Dal (1KG)                       -175703.16
Atliq_Body_Milk_Nourishing_Lotion (120ML)    -216170.00
Atliq_Doodh_Kesar_Body_Lotion (200ML)        -331740.00
Atliq_Fusion_Container_Set_of_3              -768995.00
Atliq_Sonamasuri_Rice (10KG)                -1387653.00
Name: IR, dtype: float64